In [13]:
import re
import sys
from time import time, sleep
from pprint import pprint
from functools import partial

import numpy as np
import pandas as pd
import optuna

FILENAME = "main.cpp"

In [9]:
!ls ../A/tester

README_en.html	input_1.txt  input_5.txt  input_9.txt		 testcase.py
README_ja.html	input_2.txt  input_6.txt  judge.py
generator.py	input_3.txt  input_7.txt  sample_submission.cpp
input_0.txt	input_4.txt  input_8.txt  sample_submission.py


In [14]:
# テストケース生成
SEED = 0
N_TESTCASES = 1000

!mkdir in
for i in range(N_TESTCASES):
    filename = f"{i:04d}.txt"
    seed = SEED + i
    !python3 ../A/tester/generator.py {seed} > ./in/{filename}
!ls in

mkdir: cannot create directory ‘in’: File exists
0000.txt  0125.txt  0250.txt  0375.txt	0500.txt  0625.txt  0750.txt  0875.txt
0001.txt  0126.txt  0251.txt  0376.txt	0501.txt  0626.txt  0751.txt  0876.txt
0002.txt  0127.txt  0252.txt  0377.txt	0502.txt  0627.txt  0752.txt  0877.txt
0003.txt  0128.txt  0253.txt  0378.txt	0503.txt  0628.txt  0753.txt  0878.txt
0004.txt  0129.txt  0254.txt  0379.txt	0504.txt  0629.txt  0754.txt  0879.txt
0005.txt  0130.txt  0255.txt  0380.txt	0505.txt  0630.txt  0755.txt  0880.txt
0006.txt  0131.txt  0256.txt  0381.txt	0506.txt  0631.txt  0756.txt  0881.txt
0007.txt  0132.txt  0257.txt  0382.txt	0507.txt  0632.txt  0757.txt  0882.txt
0008.txt  0133.txt  0258.txt  0383.txt	0508.txt  0633.txt  0758.txt  0883.txt
0009.txt  0134.txt  0259.txt  0384.txt	0509.txt  0634.txt  0759.txt  0884.txt
0010.txt  0135.txt  0260.txt  0385.txt	0510.txt  0635.txt  0760.txt  0885.txt
0011.txt  0136.txt  0261.txt  0386.txt	0511.txt  0636.txt  0761.txt  0886.txt
0012.txt  0137.

In [5]:
!cp ../answer/main.cpp ./

In [7]:
#!g++ {FILENAME} -std=gnu++17 -Wall -Wextra -O2 -DONLINE_JUDGE -o {FILENAME}.out
!g++ {FILENAME} -std=gnu++17 -Wall -Wextra -O2 -o {FILENAME}.out

main.cpp:49: warning: ignoring #pragma clang attribute [-Wunknown-pragmas]
   49 | #pragma clang attribute push (__attribute__((target("arch=skylake"))),apply_to=function)
      | 
main.cpp:1207: warning: ignoring #pragma clang attribute [-Wunknown-pragmas]
 1207 | #pragma clang attribute pop
      | 
main.cpp: In function ‘void Solve()’:
main.cpp:1118:72: warning: missing initializer for member ‘Solve()::Node::action’ [-Wmissing-field-initializers]
 1118 |   node_buffer.push({ state_buffer[0].score, nullptr, &state_buffer[0] });
      |                                                                        ^
main.cpp: In instantiation of ‘HashMap<T, size, KeyType>::HashMap() [with T = double; int size = 1048576; KeyType = long long unsigned int]’:
main.cpp:535:26:   required from here
main.cpp:477:30: warning: suggest parentheses around ‘-’ in operand of ‘&’ [-Wparentheses]
  477 |   static_assert((size & size - 1) == 0, "not pow of 2");
      |                         ~~~~~^~~
main.c

In [16]:
!mkdir out

In [20]:
%%time
# 実行テスト
!./{FILENAME}.out < ./in/0000.txt > ./out/0000.txt 2> /dev/null

CPU times: user 33.9 ms, sys: 9.53 ms, total: 43.4 ms
Wall time: 1.81 s


In [19]:
# 採点テスト
!python3 ../A/tester/judge.py ./in/0000.txt ./out/0000.txt

score:4403295


In [ ]:
N_TESTCASES

In [28]:
from multiprocessing import Pool

scores = []
times = []


def parallel(i):
    i = f"{i:04d}"
    t0 = time()
    !./{FILENAME}.out < ./in/{i}.txt > ./out/{i}.txt 2> /dev/null
    score = !python3 ../A/tester/judge.py ./in/{i}.txt ./out/{i}.txt
    t = time() - t0
    print(i, f"time:{t:.4f}", *score)
    score = int(score[0].split(":")[-1])
    return score, t

with Pool(4) as p:
    for score, t in p.map(parallel, range(N_TESTCASES)):
        scores.append(score)
        times.append(t)

np.mean(scores) / 1e6, np.std(scores) / 1e6

0126 time:3.5508 score:4247544
0000 time:3.8235 score:4403295
0189 time:3.9175 score:4425320
0063 time:4.1311 score:4775773
0001 time:3.47420127  time:3.7509 score:3914844
score:4138198
0064 time:3.3522 score:4247499
0190 time:3.6358 score:3894617
0002 time:3.4217 score:3967051
0128 time:3.5646 score:4205344
0065 time:3.4913 score:4136730
0191 time:3.4908 score:4216036
0003 time:3.1438 score:3705297
0066 time:3.2392 score:3518956
0192 time:3.2684 score:3727170
0129 time:3.6071 score:4530203
0004 time:3.7005 score:4346423
0130 time:3.3385 score:3788660
0067 time:3.6821 score:3911387
0193 time:3.6630 score:3885779
0131 time:3.4602 score:4031158
0068 time:3.6491 score:4240204
0194 time:3.7240 score:4016013
0005 time:4.1793 score:4044572
0132 time:3.0790 score:4105420
0006 time:3.1954 score:4034835
0195 time:3.3633 score:4258752
0069 time:3.6295 score:4400872
0133 time:3.3452 score:4465437
0070 time:3.0613 score:3992139
0007 time:3.5872 score:4493938
0196 time:3.9775 score:4729326
0134 tim

0255 time:3.4979 score:4071482
0318 time:3.3449 score:4032097
0381 time:3.8482 score:4433218
0444 time:3.9468 score:4357179
0256 time:3.8111 score:3884050
0319 time:3.7838 score:4530639
0382 time:3.8217 score:4366542
0445 time:3.9023 score:4280122
0257 time:3.6214 score:3769614
0320 time:3.5215 score:4444050
0383 time:3.5947 score:4297179
0446 time:3.1604 score:3997117
0258 time:3.6902 score:4338228
0321 time:3.6060 score:4680059
0384 time:3.9819 score:4521668
0447 time:3.6075 score:4481858
0322 time:3.4866 score:4330710
0259 time:3.6000 score:4609907
0385 time:3.3819 score:4767612
0448 time:3.4809 score:3767789
0323 time:3.4901 score:3911163
0260 time:3.5069 score:3527602
0386 time:3.4440 score:3991748
0449 time:3.0951 score:3912004
0324 time:3.5976 score:3422541
0387 time:3.6703 score:4393125
0261 time:4.0759 score:4565553
0450 time:3.5047 score:4523000
0325 time:3.1906 score:3689745
0388 time:3.2330 score:3857548
0262 time:3.8285 score:4123261
0451 time:3.3633 score:3819598
0326 tim

0510 time:3.3585 score:3343438
0573 time:3.4710 score:4314386
0699 time:3.5130 score:4944993
0636 time:3.7376 score:4471138
0511 time:3.1896 score:3785006
0574 time:3.5073 score:4144555
0700 time:3.6592 score:4120123
0637 time:3.5833 score:4435673
0512 time:3.0876 score:4090541
0575 time:3.5296 score:4019965
0638 time:3.1939 score:3299081
0701 time:3.8347 score:5213380
0513 time:3.8412 score:4121840
0576 time:3.4581 score:3566282
0639 time:3.2274 score:3865881
0702 time:3.3190 score:3800841
0514 time:3.1082 score:3923129
0577 time:3.5191 score:4055047
0640 time:3.7186score:4630252 
0703 time:3.0510 score:3907473
0515 time:3.3790 score:4107000
0578time:3.4085  score:3769109
0641 time:3.3552 score:3850927
0704 time:3.5040 score:4562427
0516 time:3.1860 score:4330734
0579 time:3.2059 score:4003910
0705 time:3.3391 score:4058853
0642 time:3.5554 score:4353856
0517 time:3.5829 score:3390486
0580 time:3.1324 score:3723998
0643 time:3.2147 score:4061994
0706 time:3.5292 score:4260706
0518 tim

0890 time:3.2320 score:4027297
0765 time:3.2268 score:4048731
0954 time:3.5540 score:4349256
0829 time:3.5597 score:4517015
0766 time:2.8950 score:4184284
0891 time:3.3619 score:3756340
0955 time:3.1525 score:3899135
0767 time:3.5292 score:4470490
0830 time:3.69250892  score:4472381
time:3.3745 score:4577503
0956 time:3.4457 score:4085837
0893 time:2.8973 score:4503945
0831 time:3.3870 score:4287826
0768 time:3.8786 score:4767349
0957 time:3.5925 score:3400135
0894 time:3.4870 score:4325729
0769 time:2.7979 score:3924908
0832 time:3.3588 score:4400616
0958 time:3.5411 score:4299452
0833 time:3.1729 score:4024360
0770 time:3.4939 score:4234496
0895 time:3.7929 score:4493854
0771 time:3.2486 score:4131766
0834 time:3.4254 score:4574071
0959 time:3.7028 score:3701292
0896 time:3.5512 score:4281773
0772 time:3.2024 score:3812601
0960 time:3.2787 score:4173188
0835 time:3.6958 score:4446165
0897 time:3.4964 score:4281219
0773 time:3.1962 score:3439615
0836 time:3.1565 score:4368018
0961 tim

(4.197495599000001, 0.3478997123473835)